In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,confusion_matrix, f1_score, precision_score, recall_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
import warnings
warnings.filterwarnings('ignore')



In [3]:
# eda_columns=['article_id', 'article_id_flag', 'clean', 'clean_comment_text',
#        'comment_text', 'count_ip', 'count_links', 'id', 'identity_hate',
#        'insult', 'ip', 'label_num', 'link', 'num_char', 'num_char_norm',
#        'num_words', 'num_words_norm', 'obscene', 'punctuation',
#        'punctuation_norm', 'sent_num', 'sent_num_norm', 'severe_toxic',
#        'stop_words', 'stop_words_norm', 'threat', 'title', 'title_norm',
#        'toxic', 'unique', 'unique_norm', 'upper', 'upper_norm', 'username',
#        'username_nums']
# non_eda_columns=['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
#        'insult', 'identity_hate', 'clean', 'label_num', 'sent_num',
#        'sent_num_norm', 'num_words', 'num_words_norm', 'num_char',
#        'num_char_norm', 'stop_words', 'stop_words_norm', 'punctuation',
#        'punctuation_norm', 'upper', 'upper_norm', 'title', 'title_norm',
#        'unique', 'unique_norm', 'ip', 'count_ip', 'link', 'count_links',
#        'article_id', 'article_id_flag', 'username', 'username_nums',
#        'clean_comment_text']
# test_columns=['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
#        'identity_hate', 'comment_text', 'sent_num', 'sent_num_norm',
#        'num_words', 'num_words_norm', 'num_char', 'num_char_norm',
#        'stop_words', 'stop_words_norm', 'punctuation', 'punctuation_norm',
#        'upper', 'upper_norm', 'title', 'title_norm', 'unique', 'unique_norm',
#        'ip', 'count_ip', 'link', 'count_links', 'article_id',
#        'article_id_flag', 'username', 'username_nums', 'clean_comment_text']

# train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/non_eda_train.txt",names=non_eda_columns,header=0)
# eda_train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/eda_train.txt",names=eda_columns,header=0)
# test=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_test_labels.txt",names=test_columns,header=0)

# # train=train.head(1000)
# # test=test.head(1000)
c_t=['clean', 'clean_comment_text', 'id', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate', 'comment_text', 'sent_num',
       'sent_num_norm', 'num_words', 'num_words_norm', 'num_char',
       'num_char_norm', 'stop_words', 'stop_words_norm', 'punctuation',
       'punctuation_norm', 'upper', 'upper_norm', 'title', 'title_norm',
       'unique', 'unique_norm', 'ip', 'count_ip', 'link', 'count_links',
       'article_id', 'article_id_flag', 'username', 'username_nums',
       'clean_comment_text_y']
test=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_eda_test.txt",names=c_t,header=0)

c=['clean_x', 'clean_comment_text', 'id', 'comment_text', 'toxic',
       'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate',
       'clean_y', 'label_num', 'sent_num', 'sent_num_norm', 'num_words',
       'num_words_norm', 'num_char', 'num_char_norm', 'stop_words',
       'stop_words_norm', 'punctuation', 'punctuation_norm', 'upper',
       'upper_norm', 'title', 'title_norm', 'unique', 'unique_norm', 'ip',
       'count_ip', 'link', 'count_links', 'article_id', 'article_id_flag',
       'username', 'username_nums', 'clean_comment_text_y']
train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_eda_train.txt",names=c,header=0)

In [4]:
embedding_file="/Users/jixusheng/Documents/kaggle/glove.6B.50d.txt"
#embedding_file="/Users/jixusheng/Documents/kaggle/crawl-300d-2M.vec"
#embedding_file="/Users/jixusheng/Documents/kaggle/GoogleNews-vectors-negative300.bin"

In [5]:
X_train=train["clean_comment_text"].fillna("fillna").values
#X_train=eda_train["clean_comment_text"].fillna("fillna").values

In [6]:
y_train =train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
#y_train =eda_train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [7]:
X_test = test["clean_comment_text"].fillna("fillna").values

In [8]:
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [9]:
#embed_size = 50 # if FastText and Word2Vec
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [10]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [11]:
###glove and fast text
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embedding_file))

####word2vec
# import gensim.models.keyedvectors as word2vec
# word2vecDict = word2vec.KeyedVectors.load_word2vec_format(embedding_file, binary=True)
# embeddings_index = dict()
# for word in word2vecDict.wv.vocab:
#             embeddings_index[word] = word2vecDict.word_vec(word)

In [12]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [13]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred,average='macro')
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))


In [14]:
def get_model():    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [15]:
model = get_model()

In [16]:
batch_size = 256
epochs = 3

In [17]:
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)

# X_tra=x_train
# y_tra=y_train
# X_val=x_test
# y_val=y_test
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

In [18]:
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=2)



Train on 757929 samples, validate on 39891 samples
Epoch 1/3
 - 3614s - loss: 0.0595 - accuracy: 0.9794 - val_loss: 0.0428 - val_accuracy: 0.9839

 ROC-AUC - epoch: 1 - score: 0.985393 

Epoch 2/3
 - 3566s - loss: 0.0411 - accuracy: 0.9843 - val_loss: 0.0376 - val_accuracy: 0.9856

 ROC-AUC - epoch: 2 - score: 0.990196 

Epoch 3/3
 - 3525s - loss: 0.0362 - accuracy: 0.9859 - val_loss: 0.0343 - val_accuracy: 0.9870

 ROC-AUC - epoch: 3 - score: 0.992144 

